In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")


DEVICE = torch.device("cuda:1")
# DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)
print(DEVICE)

Is CUDA supported by this system? True
CUDA version: 12.1
cuda:1


In [4]:
# !pip install transformers
# !pip install evaluate
# !pip install rouge
# !pip install spacy


import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from nltk.translate import meteor
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

2024-05-03 21:51:16.913084: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 21:51:16.954524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 21:51:17.687046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    df_multi = []
    df_phrase = []
    df_passage = []
    with open('/DATA1/ai20btech11028/vojes_nlp/Data/' + file_name, encoding='utf-8') as f:
        for i in f:
            i = json.loads(i)
                
            tweet = i['postText']
            article_title = i['targetTitle']
            article = ' '.join(i['targetParagraphs'])
            spoilers = i['spoiler']
            label = i['tags']
            keywords = i['targetKeywords']
            positions = i['spoilerPositions']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'phrase' or label == 'phrases':
                label = 0
            elif label == 'multi':
                label = 2
            else:
                label = 1
                
            
            df += [{'question': tweet , 'context': i['targetParagraphs'], 'article': article_title + article, 'spoiler':spoilers,
                    'labels': label, 'keywords':keywords, 'spoilerPositions':positions}]

    return pd.DataFrame(df)
            
    
# # test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')
# print(train_dataset)

In [6]:
from datasets import load_dataset

dataset = load_dataset("MateuszW/generated_questions")

genQue_train = dataset['train']['generated_questions']
genQue_test = dataset['validation']['generated_questions']
# print(genQue_train)

In [7]:
train_dataset["generated_que"] = list(genQue_train)
validation_dataset["generated_que"] = list(genQue_test)
train_dataset

,question,context,article,spoiler,labels,keywords,spoilerPositions,generated_que
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",[how about that morning we go throw?],1,"new england patriots, ricky doyle, top stories,","[[[3, 151], [3, 186]]]",What did Tom Brady do instead of having dinner...
1,NASA sets date for full recovery of ozone hole,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,[2070],0,"ozone layer,ozone hole determined by weather,M...","[[[0, 0], [0, 4]]]",What is the date that NASA has set for the ful...
2,This is what makes employees happy -- and it's...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,[intellectual stimulation],0,"employee happiness money,employee happiness in...","[[[1, 186], [1, 210]]]",What makes employees happy?
3,Passion is overrated — 7 work habits you need ...,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",[Purpose connects us to something bigger and i...,2,"business, work-life, careers","[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",What are the 7 work habits that are considered...
4,The perfect way to cook rice so that it's perf...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,[in a rice cooker],0,"Quora,users,share,perfect,way,cook,rice","[[[5, 60], [5, 76]]]",What is the best way to cook rice so that it's...
...,...,...,...,...,...,...,...,...
3195,Has Facebook's video explosion completely shak...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",[it hasn’t necessarily taken the wind out of Y...,1,"Facebook,web video,web video ads,YouTube","[[[7, 50], [7, 118]]]",Has Facebook's video explosion completely shak...
3196,Cop Is Eating At A Chili's When Teen Hands Him...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,"[It read, ""Thanks for keeping us safe.""]",1,None,"[[[0, 317], [0, 355]]]",What did the teen give the cop?
3197,5 popular myths about visible signs of aging t...,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,[1. Anti-wrinkle creams will erase the fine li...,2,,"[[[6, 0], [6, 73]], [[10, 0], [10, 109]], [[14...",What are the 5 popular myths about visible sig...
3198,You need to see this Twitter account that pred...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",[@beyoncefan666],0,,"[[[3, 55], [3, 69]]]",What is the purpose of the Twitter account men...


In [44]:
PROMPT = {
        0: (
            "Below is an question paired with a context for which generate answer."
            "Write an answer as short as possible (max 5 words). Use only words from context.\n\n"
            "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer: [MASK]\n"
        ),
        "passage": (
            "Below is an question paired with a context for which generate a passage  answer."
            "Write the most suitable answer which will be from one to three sentences. Use only words from context.\n\n"
            "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer: [MASK]\n"
        ),
        "multi": (
            "Below is an question paired with a context for which generate answer."
            "Write an answer which is multi part that means it contains multiple "
            "phrase or sentences from given text. The answers are spanned through paragraphs, check for all answers. Use only words from context.\n\n"
            "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:[MASK]\n"
        ),
        }

print(PROMPT[0])

Below is an question paired with a context for which generate answer.Write an answer as short as possible (max 5 words). Use only words from context.

### Question:
{question}

### Context:
{context}

### Answer:



In [9]:
TOKENIZER = T5TokenizerFast.from_pretrained("google-t5/t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base", return_dict=True).to(DEVICE)

OPTIMIZER = Adam(MODEL.parameters(), lr=0.0001)
Q_LEN = 256   # Question Length
T_LEN = 256  # Target Length
BATCH_SIZE = 16
DEVICE = "cuda:1"

In [10]:
# Extracting context, question, and answers from the dataset

def prepare_data(train_dataset):
    articles = []
    
    for i in range(len(train_dataset)):
        clickbait = train_dataset.iloc[i]
        # if clickbait['labels'] == 1:
        context = clickbait['article']
        question = clickbait['generated_que']
        label = clickbait['labels']
        answer = ' '.join(clickbait['spoiler'])

        inputs = {"context": context, "question": question, "answer": answer, "label":label}

        articles.append(inputs)

    return articles

In [11]:
data = prepare_data(train_dataset)

# Create a Dataframe
data = pd.DataFrame(data)

In [12]:
data

,context,question,answer,label
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",What did Tom Brady do instead of having dinner...,how about that morning we go throw?,1
1,Hole In Ozone Layer Expected To Make Full Reco...,What is the date that NASA has set for the ful...,2070,0
2,Intellectual Stimulation Trumps Money For Empl...,What makes employees happy?,intellectual stimulation,0
3,"‘Follow your passion’ is wrong, here are 7 hab...",What are the 7 work habits that are considered...,Purpose connects us to something bigger and in...,2
4,Revealed: The perfect way to cook rice so that...,What is the best way to cook rice so that it's...,in a rice cooker,0
...,...,...,...,...
3195,"Facebook Video Surging, But YouTube Still Offe...",Has Facebook's video explosion completely shak...,it hasn’t necessarily taken the wind out of Yo...,1
3196,Cop is eating at Chili's when teen hands him f...,What did the teen give the cop?,"It read, ""Thanks for keeping us safe.""",1
3197,5 popular myths about visible signs of aging t...,What are the 5 popular myths about visible sig...,1. Anti-wrinkle creams will erase the fine lin...,2
3198,"WTF, It Looks Like This Twitter Account ""Predi...",What is the purpose of the Twitter account men...,@beyoncefan666,0


In [13]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']
        self.label = self.data['label']
        
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]
        label = self.label[idx]

        PROMPT = {
        0: (
            "Below is an question paired with a context for which generate answer."
            "Write an answer as short as possible (max 5 words). Use only words from context.\n\n"
            "### The clickbait Question:\n{questions}\n\n### The article Context:\n{contexts}\n\n### The spoiler of the clickbait is [MASK]\n"
        ),
        1: (
            "Below is an question paired with a context for which generate a answer."
            "Write a complete answer  which will be from one to three sentences. Complete the answer properly. Use only words from context.\n\n"
            "### The clickbait Question:\n{questions}\n\n### The article Context:\n{contexts}\n\n### The spoiler of the clickbait is [MASK]\n"
        ),
        2: (
            "Below is an question paired with a context for which generate answer."
            "Write the spoiler which is multi part that means it contains multiple "
            "phrase or sentences as spoilers from given text. The spoilers are spanned through paragraphs, check for all spoilers. Use only words from context.\n\n"
           "### The clickbait Question:\n{questions}\n\n### The article Context:\n{contexts}\n\n### The spoiler of the clickbait is [MASK]\n"
        ),
        }


        prompt = PROMPT[label].format(questions=question, contexts=context)
        
        question_tokenized = self.tokenizer(prompt, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length", 
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)
        
        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100
        
        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [14]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [15]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

epochs = 20

for epoch in range(epochs):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1
    
    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1
        
    print(f"{epoch+1}/{epochs} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:19<00:00,  2.09it/s]


1/20 -> Train loss: 2.360201086103916	Validation loss: 1.705070897936821


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:19<00:00,  2.06it/s]


2/20 -> Train loss: 2.1603990003466604	Validation loss: 1.4913622833788396


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:19<00:00,  2.02it/s]


...

18/20 -> Train loss: 1.0050651161426989	Validation loss: 0.3050339497789941


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:19<00:00,  2.07it/s]


19/20 -> Train loss: 0.9685735204073257	Validation loss: 0.2891603490382917


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:19<00:00,  2.02it/s]

20/20 -> Train loss: 0.934653268851107	Validation loss: 0.2748656421733904


In [25]:
MODEL.save_pretrained("qa_model_50_epochs")
TOKENIZER.save_pretrained("qa_tokenizer_50_epochs")

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

"('qa_tokenizer/tokenizer_config.json',\n 'qa_tokenizer/special_tokens_map.json',\n 'qa_tokenizer/spiece.model',\n'qa_tokenizer/added_tokens.json',\n'qa_tokenizer/tokenizer.json')"

In [27]:
def predict_answer(context, question, label, ref_answer=None):
    PROMPT = {
        0: (
            "Below is an question paired with a context for which generate answer."
            "Write an answer as short as possible (max 5 words). Use only words from context.\n\n"
            "### Question:\n{questions}\n\n### Context:\n{contexts}\n\n### Answer:\n"
        ),
        1: (
            "Below is an question paired with a context for which generate a answer."
            "Write a complete answer which will be from one to three sentences. Use only words from context.\n\n"
            "### Question:\n{questions}\n\n### Context:\n{contexts}\n\n### Answer:\n"
        ),
        2: (
            "Below is an question paired with a context for which generate answer."
            "Write an answer which is multi part that means it contains multiple "
            "phrase or sentences from given text. The answers are spanned through paragraphs, check for all answers. Use only words from context.\n\n"
            "### Question:\n{questions}\n\n### Context:\n{contexts}\n\n### Answer:\n"
        ),
        }

    # print(label)
    prompt = PROMPT[label].format(questions=question, contexts=context)
        
    inputs = TOKENIZER(prompt, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
    
    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)
  
    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)
    
    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer], 
                            references=[ref_answer])
        bert = evaluate.load("bertscore")

        bert_score = bert.compute(predictions=[predicted_answer], references=[ref_answer], lang='en')
        ref_token = nltk.word_tokenize(predicted_answer)
        pre_token = nltk.word_tokenize(ref_answer)
        # print(ref_token)
        # print(pre_token)
        meteor_score = meteor([ref_token], pre_token)

        ref_token = nltk.word_tokenize(predicted_answer)
        pre_token = nltk.word_tokenize(ref_answer)
        # print(ref_token)
        # print(pre_token)
        meteor_score = meteor([ref_token], pre_token)
    
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer, 
            "Predicted Answer: ": predicted_answer, 
            "BLEU Score: ": score,
            "Bert Score: ": bert_score ,
            "Meteor Score: ": meteor_score
        }
    else:
        return predicted_answer

In [28]:
validation_dataset

,question,context,article,spoiler,labels,keywords,spoilerPositions,generated_que
0,Five Nights at Freddy’s Sequel Delayed for Wei...,[Five Nights at Freddy’s creator Scott Cawthon...,Five Nights at Freddy’s Sequel Delayed for Wei...,[some of the plot elements are so disturbing t...,1,None,"[[[2, 158], [2, 236]]]",What is the reason for the delay of the sequel...
1,Why Arizona Sheriff Joe Arpaio’s fate could ha...,[© REUTERS/Laura Segall Maricopa County Sherif...,Why Arizona Sheriff Joe Arpaio’s fate could ha...,"[""intentionally"", could transform a court case...",2,None,"[[[0, 197], [0, 212]], [[0, 215], [0, 328]]]",What is the word that could determine the fate...
2,Here’s how much you should be tipping your hai...,[Here’s how much you should be tipping your ha...,Here’s how much you should be tipping your hai...,[20%],0,,"[[[3, 58], [3, 61]]]",How much should you be tipping your hairdresser?
3,"""Harry Potter"" alums reunite for new movie",[The mythology of punk music's evolution can b...,"Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...","[Alan Rickman & Rupert Grint, CBGB]",2,"Alan Rickman,Hilly Kristal,new rupert grint mo...","[[[-1, 0], [-1, 27]], [[0, 98], [0, 102]]]","What is the name of the new movie in which ""Ha..."
4,A man swallowed a microSD card and you won't b...,[PetaPixel is one of my favorite blogs. The wr...,Man swallowed a microSD card and you won't bel...,[a man who swallowed a 64GB microSD card and t...,1,None,"[[[1, 34], [1, 108]]]",What happened next?
...,...,...,...,...,...,...,...,...
795,This is what happens when you leave a hotel cl...,[Instead of encountering a mound of dirty towe...,This Is What Happens When You Leave A Hotel Cl...,[The video below shows the stunned cleaner ini...,1,"givebackfilms,give back films,video,random act...","[[[3, 0], [3, 150]]]",What happens when you leave a hotel cleaner a ...
796,This Texas GOP elector announces that he won't...,[A Republican elector in Texas says he will no...,Texas GOP elector announces he won't vote for ...,[Christopher Suprun],0,"donald trump, texas, electoral college, faithl...","[[[1, 45], [1, 63]]]",What is the name of the Texas GOP elector who ...
797,This beauty queen cured her acne with one diet...,[Her inspirational journey is encouraging othe...,UK beauty queen cured her severe acne with one...,"[Rachel Crawley, High fat vegan plant based di...",2,"acne, Skincare, beauty, Beauty pageant, Dermat...","[[[2, 144], [2, 158]], [[6, 56], [6, 124]]]",What diet change cured the beauty queen's acne?
798,WikiLeaks' Julian Assange Reported Dead,"[On 16 October 2016, WikiLeaks posted a series...","WikiLeaks’ Julian Assange Isn’t Dead, Just Off...",[Julian Assange’s internet link has been inten...,1,"dead man's switch, julian assange, wikileaks","[[[11, 0], [11, 78]]]",What is the meaning of the sentence?


In [31]:
_all = {}
arr = [{} for i in range(3)]
bleu = 0
meteor_int = 0
bert = 0
count = 0

for i in range(100):
    print('--------', i)
    question = validation_dataset.iloc[i]['generated_que']
    context = validation_dataset.iloc[i]['article']
    answer = ' '.join(validation_dataset.iloc[i]['spoiler'])
    label = validation_dataset.iloc[i]['labels']

    # print(validation_dataset.iloc[i]['labels'])
    ans = predict_answer(context, question, label, answer)
    _all['bleu'] = _all.get('bleu', 0) + ans["BLEU Score: "]['google_bleu']
    _all['bert'] = _all.get('bert', 0) + ans['Bert Score: ']['precision'][0]
    _all['meteor'] = _all.get('meteor', 0) + ans['Meteor Score: ']


    type = validation_dataset.iloc[i]['labels']
    arr[type]['bleu'] = arr[type].get('bleu',0) + ans["BLEU Score: "]['google_bleu']
    arr[type]['bert'] = arr[type].get('bert',0) + ans["Bert Score: "]['precision'][0]
    arr[type]['meteor'] = arr[type].get('meteor',0) + ans["Meteor Score: "]
    arr[type]['count'] = arr[type].get('count',0) + 1
        
        
    
    print(ans)
    # if validation_dataset.iloc[i]['labels'] == 2:
    #     continue
    count += 1
    bleu += ans["BLEU Score: "]['google_bleu']
    bert += ans["Bert Score: "]['precision'][0]
    meteor_int += ans["Meteor Score: "]
    
    

print(" All " + str(_all['bert']/100))
print(" Phrase " + str(arr[0]['bert']/arr[0]['count']))
print(" Passage " + str(arr[1]['bert']/arr[1]['count']))
print(" Multi " + str(arr[2]['bert']/arr[2]['count']))


    

-------- 0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the reason for the delay of the sequel to Five Nights at Freddy's?
{'Reference Answer: ': 'some of the plot elements are so disturbing that they are making him feel sick', 'Predicted Answer: ': 'the sequel is due to a computer glitch that occurred during the development of the game.', 'BLEU Score: ': {'google_bleu': 0.06451612903225806}, 'Bert Score: ': {'precision': [0.8697110414505005], 'recall': [0.8611655235290527], 'f1': [0.8654171824455261], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.22321428571428575}
-------- 1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the word that could determine the fate of Arizona Sheriff Joe Arpaio?
{'Reference Answer: ': '"intentionally" could transform a court case against Phoenix-area Sheriff Joe Arpaio from civil charges to a criminal prosecution', 'Predicted Answer: ': 'On May 31, Snow will determine the civil penalties and examine the motives of the three defendants', 'BLEU Score: ': {'google_bleu': 0.013513513513513514}, 'Bert Score: ': {'precision': [0.8624863624572754], 'recall': [0.8442448377609253], 'f1': [0.8532680869102478], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.02747252747252747}
-------- 2


...

Question: 
 What is the disturbing reason?
{'Reference Answer: ': 'They are soaked in a bath of chlorine', 'Predicted Answer: ': 'They aren’t small carrots. No, baby carrots are big carrots that', 'BLEU Score: ': {'google_bleu': 0.043478260869565216}, 'Bert Score: ': {'precision': [0.8415586352348328], 'recall': [0.8600798845291138], 'f1': [0.8507184982299805], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.06993006993006992}
-------- 98


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the iPhone 5C number that Apple doesn't want you to know?
{'Reference Answer: ': 'preorder figure was 2 million', 'Predicted Answer: ': '2 million', 'BLEU Score: ': {'google_bleu': 0.21428571428571427}, 'Bert Score: ': {'precision': [0.8882178664207458], 'recall': [0.8407328128814697], 'f1': [0.8638232350349426], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.8152173913043478}
-------- 99


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the reason the mother glued pennies to her daughter's shoes?
{'Reference Answer: ': 'make-shift tap shoes', 'Predicted Answer: ': 'they were too expensive', 'BLEU Score: ': {'google_bleu': 0.0}, 'Bert Score: ': {'precision': [0.84417724609375], 'recall': [0.8330268263816833], 'f1': [0.838564932346344], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.0}
 All 0.9004877907037735
 Phrase 0.9323202384279129
 Passage 0.8705274760723114
 Multi 0.8753575619898344


In [32]:
print(bleu/count)
print(bert/count)
print(meteor_int/count)

0.3320358861768656
0.9004877907037735
0.39203029255122696


In [33]:
print("----------       bleu-score           bert-score             meteor-score")
print(" All      ", _all['bleu']/100, "  ", _all['bert']/100, " ", _all['meteor']/100)
print(" Phrase   ", arr[0]['bleu']/arr[0]['count'], "  ", arr[0]['bert']/arr[0]['count'],  "  ", arr[0]['meteor']/arr[0]['count'])
print(" Passage  ", arr[1]['bleu']/arr[1]['count'], "   ", arr[1]['bert']/arr[1]['count'], " ", arr[1]['meteor']/arr[1]['count'])
print(" Multi    ", arr[2]['bleu']/arr[2]['count'], "  ", arr[2]['bert']/arr[2]['count'], " ", arr[2]['meteor']/arr[2]['count'])

----------       bleu-score           bert-score             meteor-score
 All       0.3320358861768656    0.9004877907037735   0.39203029255122696
 Phrase    0.5610942249240123    0.9323202384279129    0.5462192181787294
 Passage   0.14187682187373252     0.8705274760723114   0.2581797331172759
 Multi     0.10570253171321424    0.8753575619898344   0.25013763551237
